# Thymus ageing atlas: Cell communication within the cortex

In [ ]:
import os
import sys
import session_info
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad
import hdf5plugin

import liana as li
from liana.method import cellphonedb
import muon as mu
import mofax as mofa

import warnings
warnings.filterwarnings('ignore', category=ad.ImplicitModificationWarning)

# Add repo path to sys path (allows to access scripts and metadata from repo)
repo_path,_ = os.path.split(os.path.split(os.getcwd())[0])
repo_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts')

# Autoreload custom scripts
%load_ext autoreload
%autoreload 2

# Define paths
plots_path = f'{repo_path}/plots/'
data_path = f'{repo_path}/data/'
model_path = os.path.join(repo_path, 'models')
general_data_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/data'

print('Dir for plots: {}'.format(plots_path))
print('Dir for data: {}'.format(data_path))

# Formatting
from matplotlib import font_manager
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
plt.style.use('/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
from plotting.utils import plot_grouped_boxplot, calc_figsize, thyAgeing_colors

In [ ]:
# Define columns
col_cell_type_broad = 'taa_l3'
col_cell_type_fine = 'taa_l4'
col_cell_type_broad_levels = get_ct_levels(col_cell_type_broad, taa_l1 = ['T', 'NK'])
col_cell_type_fine_levels = get_ct_levels(col_cell_type_fine, taa_l1 = ['T', 'NK'])
col_age_group = 'age_group'
col_age_group_levels = eval(f'{col_age_group}_levels')
col_sample = 'donor'

In [ ]:
# Load adata
object_version = 'v5_2025-04-03'
adata = ad.read_h5ad(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_{object_version}.zarr')

# Add new annotations to adata
ct_anno = pd.read_csv(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_v4_2025-02-04_curatedAnno_v10.csv', index_col = 0)
for c in ct_anno.columns:
    if c in adata.obs.columns:
        adata.obs.drop(c, axis = 1, inplace = True)
adata.obs = adata.obs.join(ct_anno)

# Filter data (only include annotated cells)
adata = adata[(adata.obs['anno_status'] == 'include') & (adata.obs['qc_status'] == 'PASS'),:]

# Update metadata
latest_meta_path = get_latest_version(dir = f'{general_data_path}/metadata', file_prefix='Thymus_ageing_metadata')
latest_meta = pd.read_excel(latest_meta_path)
update_obs(adata, latest_meta, on = 'index', ignore_warning = True)

adata

## Early thymocyte development (cortex)

In [ ]:
# List cell types of interest
ctoi = adata.obs[adata.obs['taa_l2'].isin(['T_dev', 'T_predev'])]['taa_l4'].unique().tolist() + ['B_dev_thy', 'pDC']
ctoi.extend(adata.obs[adata.obs['taa_l1'].isin(['TEC', #'Fb', 'EC', 'B', 'Mural', 'Mac', 'Mono', 'DC'
                                                ])]['taa_l4'].unique().tolist())

np.array(ctoi)

In [ ]:
# Subset adata
adata_sub = adata[adata.obs['taa_l4'].isin(ctoi),:].copy()
adata_sub.obs['cell_type'] = adata_sub.obs.apply(lambda x: 'pDC' if x['taa_l4'] == 'pDC' else x['taa_l4'] if x['taa_l1'] in ['T', 'B'] else x['taa_l3'], axis = 1)

# Remove B_dev (very few cells)
adata_sub = adata_sub[adata_sub.obs['cell_type'] != 'B_dev',:]

adata_sub.obs['cell_type'].value_counts()

In [ ]:
# Load CellPhoneDB database (v5.0.0)
ia_db = pd.read_csv(f'{data_path}/references/ccc_databases/cellphoneDB_v5.0.0_interaction_input.csv')
ia_db[['ligand', 'receptor']] = ia_db['interactors'].str.rsplit('-', n=1, expand=True)
ia_db['receptor'] = ia_db['receptor'].str.split('+')
ia_db = ia_db.explode('receptor')

# Select interacrions of interest
lroi = ['NOTCH1', 'LTB', 'LTA', 'IL7', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'IL7R', 'CXCR4', 'DLK2', 'CCL25']
interactions_oi = ia_db.loc[(ia_db['ligand'].isin(lroi)) | (ia_db['receptor'].isin(lroi)), ['ligand', 'receptor']]
interactions_oi = list(interactions_oi.itertuples(index=False, name=None))
interactions_oi += [('DLK2', 'NOTCH2')]

interactions_oi
pd.DataFrame(interactions_oi, columns=['ligand', 'receptor']).to_csv(f'{data_path}/curated/thymus_development_interactions.csv', index = False)

In [ ]:
# Get genes of interest and subset adata
moi = list(set([i[0] for i in interactions_oi] + [i[1] for i in interactions_oi]))
np.array(moi)

adata_sub = adata_sub[:, adata_sub.var.index.isin(moi)].copy()

# Log-normalise data
sc.pp.normalize_total(adata_sub, target_sum=1e4)
sc.pp.log1p(adata_sub)

In [ ]:
# Filter out donors with fewer than 100 cells
min_cells = 100
ncells_by_sample = adata_sub.obs[col_sample].value_counts().to_frame(name = 'n_cells')
print('Removing {} donors with fewer than {} cells'.format(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].shape[0], min_cells))
adata_sub = adata_sub[~adata_sub.obs[col_sample].isin(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].index)]

In [ ]:
# Level to cell type assignment
ct_anno_levels = adata_sub.obs[['cell_type', 'taa_l4', 'taa_l3', 'taa_l1']].drop_duplicates().copy()
ct_anno_levels['level'] = ct_anno_levels.apply(lambda x: 'taa_l4' if x['taa_l1'] in ['T', 'B'] else 'taa_l3', axis = 1)
ct_anno_levels = ct_anno_levels.groupby('level').agg(list)
ct_anno_levels = ct_anno_levels.to_dict()['cell_type']

taa_l4_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_fine}_byDonor_freq.csv', index_col = 0)
taa_l4_freq = taa_l4_freq.loc[taa_l4_freq[col_cell_type_fine].isin(ct_anno_levels['taa_l4']),:].rename(columns = {col_cell_type_fine: 'cell_type'})
taa_l3_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_broad}_byDonor_freq.csv', index_col = 0)
taa_l3_freq = taa_l3_freq.loc[taa_l3_freq[col_cell_type_broad].isin(ct_anno_levels['taa_l3']),:].rename(columns = {col_cell_type_broad: 'cell_type'})

ct_freq = pd.concat([taa_l4_freq, taa_l3_freq], axis = 0)
ct_freq.head()

### Cell comm analysis by sample

In [ ]:
li.mt.cellphonedb.by_sample(
    adata_sub,
    groupby='cell_type',
    #resource_name='cellphonedb', 
    sample_key=col_sample, 
    expr_prop = 0.1,
    use_raw=False,
    interactions=interactions_oi,
    n_perms=500, 
    return_all_lrs=True, 
    verbose=True, 
    n_jobs = 4
    )

In [ ]:
sample_res = adata_sub.uns["liana_res"].copy()
sample_res = sample_res.merge(adata_sub.obs[[col_sample, col_age_group]].drop_duplicates(), on = col_sample, how = 'inner')

sample_res['interaction'] = sample_res['ligand'] + ' -> ' + sample_res['receptor']
sample_res['cells'] = sample_res['source'] + ' -> ' + sample_res['target']

# Account for cell type frequency
sample_res = sample_res.merge(ct_freq[['donor', 'cell_type', 'mean_prop']], left_on = ['donor', 'source'], right_on = ['donor', 'cell_type'], how = 'left').rename(columns = {'mean_prop': 'source_prop'}).drop('cell_type', axis = 1)
sample_res = sample_res.merge(ct_freq[['donor', 'cell_type', 'mean_prop']], left_on = ['donor', 'target'], right_on = ['donor', 'cell_type'], how = 'left').rename(columns = {'mean_prop': 'target_prop'}).drop('cell_type', axis = 1)

sample_res.to_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_all_scvi_{object_version}_cortex_cellphoneDbBySample.csv', index = False)

### Differential cell communication analysis

In [ ]:
sample_res = pd.read_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_all_scvi_{object_version}_cortex_cellphoneDbBySample.csv', index_col = None)
sample_res['tot_prob'] = sample_res['source_prop'] * sample_res['target_prop'] * sample_res['ligand_props'] * sample_res['receptor_props']
sample_res.head()

In [ ]:
# Check that only expected donors/samples are missing tot_prob (the ones which are not TOT sort)
all(sample_res.loc[sample_res['tot_prob'].isna()]['donor'].unique() == np.setdiff1d(sample_res['donor'].unique(), ct_freq['donor'].unique()))

In [ ]:
# Calculate mean properties (using all samples, not just TOT sort)
mean_props = sample_res.groupby(['interaction', 'cells', 'age_group']).agg({'ligand_props' : 'mean', 'receptor_props' : 'mean','source_prop': 'mean', 'target_prop': 'mean', 'tot_prob': 'mean'}).reset_index()
mean_props = mean_props.pivot(index = ['interaction', 'cells'], columns = 'age_group', values = ['ligand_props', 'receptor_props', 'source_prop', 'target_prop','tot_prob']).reset_index()
mean_props.columns = ['_'.join(col).strip() for col in mean_props.columns.values]
mean_props = mean_props.rename(columns = {'interaction_': 'interaction', 'cells_': 'cells'})
mean_props.head()

In [ ]:
# Filter interactions
# Select interactions with at least one age group with expression > 0.1
target_age_group = ['adult', 'infant']
min_expr = 0.1

# Keeping any interaction with at least one age group with expression > in expr
#ia_to_keep = mean_props.loc[mean_props[['ligand_props_' + ag for ag in target_age_group] + ['receptor_props_' + ag for ag in target_age_group]].ge(min_expr).all(axis=1)][['interaction', 'cells']]

# Keeping interactions where L and R are both expressed at min expr in the same age group
ia_to_keep = []
for g in target_age_group:
    ia_to_keep.append(mean_props.loc[mean_props[[f'ligand_props_{g}', f'receptor_props_{g}']].ge(min_expr).all(axis=1)][['interaction', 'cells']])
ia_to_keep = pd.concat(ia_to_keep, axis = 0).drop_duplicates()

ia_to_keep.shape

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from scipy.stats import ranksums
from statsmodels.stats.multitest import multipletests

donor_n = ct_freq[['donor', col_age_group]].drop_duplicates().groupby('age_group').agg({'donor': 'count'}).to_dict()['donor']
# Define a function to process a single cell
def process_cells(ct):
    ias = ia_to_keep[ia_to_keep['cells'] == ct]['interaction'].unique()
    test_ranksum = []
    for ia in ias:
        test_data = sample_res[(sample_res['interaction'] == ia) & (sample_res['cells'] == ct)][['donor', col_age_group, 'tot_prob']].dropna(subset=['tot_prob']).copy()
        group1 = test_data[test_data['age_group'] == target_age_group[0]]['tot_prob'].to_numpy()
        if group1.shape[0] < donor_n[target_age_group[0]]:
            group1 = np.concatenate((group1, np.repeat(0, repeats=donor_n[target_age_group[0]] - group1.shape[0])))
        group2 = test_data[test_data['age_group'] == target_age_group[1]]['tot_prob'].to_numpy()
        if group2.shape[0] < donor_n[target_age_group[1]]:
            group2 = np.concatenate((group2, np.repeat(0, repeats=donor_n[target_age_group[1]] - group2.shape[0])))
        test_res = ranksums(group1, group2)
        test_ranksum.append([ia, ct, test_res[0], test_res[1]])
    test_ranksum = pd.DataFrame(test_ranksum, columns=['interaction', 'cells', 'statistic', 'pvalue'])
    test_ranksum['padj'] = multipletests(test_ranksum['pvalue'], method='fdr_bh')[1]
    return test_ranksum

# Use ThreadPoolExecutor with 4 threads to process cells in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_cells, ia_to_keep['cells'].unique()))

# Combine results
ranksums_res = pd.concat(results, axis=0)

ranksums_res.to_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_all_scvi_{object_version}_cortex_cellphoneDbBySample_ranksums_adult.csv')

ranksums_res.sort_values('statistic', ascending=False)

In [ ]:
ranksums_res = pd.read_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_all_scvi_{object_version}_cortex_cellphoneDbBySample_ranksums_adult.csv', index_col = 0)
ranksums_res[['source', 'target']] = ranksums_res['cells'].str.split(' -> ', expand=True)
ranksums_res[['ligand', 'receptor']] = ranksums_res['interaction'].str.split(' -> ', expand=True)
ranksums_res.sort_values('statistic', ascending=False)

In [ ]:
ranksums_res_filtered = ranksums_res.loc[(ranksums_res['padj'] < .05)]
ranksums_res_filtered.sort_values('statistic', ascending=False)

Add DEG and population change info:

In [ ]:
# Import DEGs
import pickle 

taa_l4_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_taa_l4_adult_vs_infant_ageEffect.pkl'
taa_l3_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_taa_l3_adult_vs_infant_ageEffect.pkl'

with open(taa_l4_degs, 'rb') as f:
    taa_l4_degs = pickle.load(f)
taa_l4_degs = {k:v for k,v in taa_l4_degs.items() if k in ct_anno_levels['taa_l4']}
taa_l4_degs = pd.concat(taa_l4_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

with open(taa_l3_degs, 'rb') as f:
    taa_l3_degs = pickle.load(f)
taa_l3_degs = {k:v for k,v in taa_l3_degs.items() if k in ct_anno_levels['taa_l3']}
taa_l3_degs = pd.concat(taa_l3_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

all_degs = pd.concat([taa_l4_degs, taa_l3_degs])
all_degs = all_degs.loc[all_degs.index.isin(moi),:]

all_degs.to_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_diffCCC_cortex_degs.csv', index = True)
all_degs.head()

In [ ]:
from scipy.cluster.hierarchy import linkage, leaves_list

df = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'log2FoldChange')
df_annot = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'padj')
df_annot = df_annot.applymap(lambda x: '*' if x < 0.5 else '')

# Perform hierarchical clustering on the columns
linkage_matrix = linkage(df.T.fillna(0), method='ward')
column_order = leaves_list(linkage_matrix)

# Perform hierarchical clustering on the rows
row_linkage_matrix = linkage(df.fillna(0), method='ward')
row_order = leaves_list(row_linkage_matrix)

# Reorder the columns of the dataframe
df = df.iloc[row_order, column_order]

# Plot the reordered heatmap
p = sns.heatmap(df, cmap=sns.blend_palette([thyAgeing_colors['teal'], 'white',thyAgeing_colors['orange']], as_cmap=True),
                center=0, vmin=-5, vmax=5, cbar_kws={'label': 'log2FC'}, xticklabels=True, yticklabels=True,
                annot=df_annot.iloc[row_order, column_order], fmt='', annot_kws={'size': 8,'weight': 'bold'},)
p.set_xlabel('Cell type')
p.set_ylabel('Gene')
p.figure.set_size_inches(calc_figsize(width=100, height=60))
p.figure.tight_layout(rect=[0, 0, 1, 0.95], pad = 0)
plt.savefig(f'{plots_path}/cellComm/cortex/thyAgeing_diffCCC_cortex_degs_heatmap_clustered.pdf', bbox_inches='tight')

In [ ]:
# Import logFC for each cell type
logfc_freq = pd.read_csv(f'{general_data_path}/analyses/freqAnalysis/thyAgeing_all_scvi_v5_2025-04-03_milo_ageGroups_medianLogFC.csv', index_col = 0)
logfc_freq = logfc_freq.loc[logfc_freq['comparison'] == 'adult_vs_infant',:]

all_freq = []
for k,v in ct_anno_levels.items():
    all_freq.append(logfc_freq.loc[(logfc_freq['anno'].isin(v)) & (logfc_freq['anno_level'] == k),:])
all_freq = pd.concat(all_freq)

all_freq.head()

In [ ]:
ranksums_res_filtered = ranksums_res.loc[(ranksums_res['padj'] < .05)]
ranksums_res_filtered = ranksums_res_filtered.merge(all_freq[['anno', 'logFC']], left_on = 'source', right_on = 'anno', how = 'left').drop('anno', axis = 1).rename(columns = {'logFC': 'source_logFC'})
ranksums_res_filtered = ranksums_res_filtered.merge(all_freq[['anno', 'logFC']], left_on = 'target', right_on = 'anno', how = 'left').drop('anno', axis = 1).rename(columns = {'logFC': 'target_logFC'})
ranksums_res_filtered = ranksums_res_filtered.merge(all_degs[['cell_type', 'log2FoldChange', 'padj']].reset_index(names = 'gene_name').rename(columns = {'log2FoldChange': 'ligand_logFC', 'padj': 'ligand_padj'}), left_on = ['ligand', 'source'], right_on = ['gene_name', 'cell_type'], how = 'left').drop(['gene_name', 'cell_type'], axis = 1)
ranksums_res_filtered = ranksums_res_filtered.merge(all_degs[['cell_type', 'log2FoldChange', 'padj']].reset_index(names = 'gene_name').rename(columns = {'log2FoldChange': 'receptor_logFC', 'padj': 'receptor_padj'}), left_on = ['receptor', 'target'], right_on = ['gene_name', 'cell_type'], how = 'left').drop(['gene_name', 'cell_type'], axis = 1)
ranksums_res_filtered['pop_driven'] = ranksums_res_filtered.apply(lambda x: abs(x['source_logFC']) >= 1.3 or abs(x['target_logFC']) >= 1.3, axis = 1)
ranksums_res_filtered['deg_driven'] = ranksums_res_filtered.apply(lambda x: abs(x['ligand_padj']) < .05 or abs(x['receptor_padj']) < .05, axis = 1)
ranksums_res_filtered.head()

In [ ]:
ranksums_res_filtered.to_csv(f'{data_path}/analyses/cellComm/cortex/thyAgeing_all_scvi_{object_version}_cortex_cellphoneDbBySample_ranksums_filtered.csv')

In [ ]:
ranksums_res_filtered[['pop_driven', 'deg_driven']].value_counts()

In [ ]:
ranksums_res_filtered.loc[ranksums_res_filtered['statistic'] < 0]

In [ ]:
# Heatmap of number of differential interactions (no signif downregulated interactions)
from scipy.cluster.hierarchy import linkage, leaves_list

df = ranksums_res_filtered.loc[ranksums_res_filtered['statistic'] > 0].copy()
df = df.groupby(['source', 'target']).size().to_frame(name='n_interactions').reset_index()
df = df.pivot(index='source', columns='target', values='n_interactions').fillna(0)

# Cluster rows and columns

row_linkage = linkage(df, method='ward')
col_linkage = linkage(df.T, method='ward')

row_order = leaves_list(row_linkage)
col_order = leaves_list(col_linkage)

df = df.iloc[row_order, col_order]

p = sns.heatmap(df, cmap='Spectral_r', annot=None, fmt='.2g', xticklabels='auto', yticklabels='auto',
                vmax = 20)
p.set_xlabel('Target cell type')
p.set_ylabel('Source cell type')
p.figure.tight_layout()
p.figure.set_size_inches(calc_figsize(width = 120, height = 120))
plt.savefig(f'{plots_path}/cellComm/cortex/thyAgeing_diffCCC_cortex_filteredSignallingUp_heatmap.pdf')

# Heatmap of number of differential interactions (no signif downregulated interactions)
df = ranksums_res_filtered.loc[ranksums_res_filtered['statistic'] < 0].copy()
df = df.groupby(['source', 'target']).size().to_frame(name='n_interactions').reset_index()
df = df.pivot(index='source', columns='target', values='n_interactions').fillna(0)

# Cluster rows and columns

row_linkage = linkage(df, method='ward')
col_linkage = linkage(df.T, method='ward')

row_order = leaves_list(row_linkage)
col_order = leaves_list(col_linkage)

df = df.iloc[row_order, col_order]

p = sns.heatmap(df, cmap='Spectral_r', annot=None, fmt='.2g', xticklabels='auto', yticklabels='auto',
                vmax = 20)
p.set_xlabel('Target cell type')
p.set_ylabel('Source cell type')
p.figure.tight_layout()
p.figure.set_size_inches(calc_figsize(width = 120, height = 120))
plt.savefig(f'{plots_path}/cellComm/cortex/thyAgeing_diffCCC_cortex_filteredSignallingDown_heatmap.pdf')

In [ ]:
# Plot ligand-receptor interactions of interest (DEGs)
sources = ['cTEC','mcTEC', #'mTECI', 'mTECII', 'mTECIII', 'TEC-EMT', 'TEC-mim'
           ]
targets = ['T_DN(early)', #'B_dev_thy', 'pDC',
           'T_DN(P)', 'T_DN(Q)', 'T_DP(P)']
ia_list = ['DLL1^NOTCH1', 'DLL4^NOTCH1', 'JAG1^NOTCH1', 'JAG2^NOTCH1','DLK2^NOTCH2', 'CXCL12^CXCR4', 'CCL25^CCR9']

df = pd.DataFrame({'interaction' : ia_list})
df[['ligand', 'receptor']] = df['interaction'].str.split('^', expand=True)  
df = df.melt(id_vars = 'interaction', value_name= 'gene_name', var_name = 'lr')
df = df.merge(all_degs.reset_index(names = 'gene_name'), on = 'gene_name', how = 'left')

# Construct dataframes for ligand and receptor
df_ligand = df.loc[(df['lr'] == 'ligand') & (df['cell_type'].isin(sources)), ['interaction', 'gene_name', 'log2FoldChange', 'padj', 'cell_type']].pivot_table(index = 'interaction', columns = 'cell_type', values = 'log2FoldChange').loc[ia_list, sources]
df_ligand_anno = df.loc[(df['lr'] == 'ligand') & (df['cell_type'].isin(sources)), ['interaction', 'gene_name', 'padj', 'cell_type']].pivot_table(index = 'interaction', columns = 'cell_type', values = 'padj').map(lambda x: '*' if x < 0.05 else '').loc[ia_list, sources]
df_receptor = df.loc[(df['lr'] == 'receptor') & (df['cell_type'].isin(targets)), ['interaction', 'gene_name', 'log2FoldChange', 'padj', 'cell_type']].pivot_table(index = 'interaction', columns = 'cell_type', values = 'log2FoldChange').loc[ia_list, targets]
df_receptor_anno = df.loc[(df['lr'] == 'receptor') & (df['cell_type'].isin(targets)), ['interaction', 'gene_name', 'padj', 'cell_type']].pivot_table(index = 'interaction', columns = 'cell_type', values = 'padj').map(lambda x: '*' if x < 0.05 else '').loc[ia_list, targets]

In [ ]:
from matplotlib import colors as mcolors
max_abs = np.percentile(abs(np.concatenate([df_ligand.values.flatten(), df_receptor.values.flatten()])), 95) # 95th percentile of absolute values

width_ratios = [len(sources), len(targets)]
fig, axes = plt.subplots(1,2, figsize=calc_figsize(width=45, height=25), 
                            gridspec_kw={'width_ratios': width_ratios, 'wspace': 0.05}
                            )

# Color map
cmap = sns.blend_palette([thyAgeing_colors['teal'], 'white', thyAgeing_colors['orange']], as_cmap=True)
norm = mcolors.TwoSlopeNorm(vcenter=0, vmin=-max_abs, vmax=max_abs)

sns.heatmap(df_ligand, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[0],
                annot=np.array(df_ligand_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[0].set_title(f'Ligand', fontweight='bold')
axes[0].set_xlabel('Cell population')
axes[0].set_ylabel('Interaction')
axes[0].tick_params(axis='x', rotation=90)
axes[0].tick_params(axis='y', rotation=0)

sns.heatmap(df_receptor, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[1],
                annot=np.array(df_receptor_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[1].set_title(f'Receptor', fontweight='bold')
axes[1].set_xlabel('Cell population')
axes[1].tick_params(axis='x', rotation=90)
axes[1].set_ylabel('')
axes[1].set_yticklabels([])
axes[1].set_yticks([])

for ax in axes:
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', length=0)

 # Add a colorbar to the right of the last heatmap
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='vertical', fraction=0.05, pad=0.0)
cbar.set_label('Log2FC', rotation=270, labelpad=5)
cbar.outline.set_visible(False)
fig.subplots_adjust(right=0.85)

plt.savefig(f'{plots_path}/cellComm/cortex/thyAgeing_diffCCC_cortex_ia_heatmap.pdf')

[Dutta,2021](https://www.sciencedirect.com/science/article/pii/S1471490621001198):
- Studies performed mainly in the mouse model system over the past two decades have firmly established an essential role for coordinated Notch and pre-TCR signaling in β-selection [20,21]. In addition to Notch and the pre-TCR, important functions for the chemokine receptor Cxcr4 and its ligand Cxcl12, as well as for morphogen signaling [Hedgehog, bone morphogenetic proteins (BMPs), and Wnt] (Box 3), and IL-7 receptor signaling (Box 4), have also been demonstrated to be important for successful passage through the β-selection checkpoint (Box 3, Box 4) [20,21]. 
- In mice, Lfng glycosylates epidermal growth factor (EGF) repeats in the Notch1 extracellular domain, which enhances binding affinity for its ligand, delta-like 4 (DL4), resulting in suppression of B cell lineage commitment in DN1 thymocytes and promotion of αβ-T cell lineage commitment in DN2/3 thymocytes [29., 30., 31., 32.]
- However, after β-selection, pre-TCR signaling inhibited Notch1 transcription by inducing the expression of the transcription factor Id3 [25]. 
- Cdkn1b is highly expressed in quiescent ‘pre-β-selection’ DN3 mouse thymocytes, but is downregulated upon initiation of β-selection [41]. 

## Late thymocyte development

In [ ]:
# List cell types of interest
ctoi = adata.obs[adata.obs['taa_l2'].isin(['T_dev', 'T_mature'])]['taa_l4'].unique().tolist() 
ctoi.remove('T_DN(P)')
ctoi.remove('T_DN(Q)')
ctoi.remove('T_DN(late)')
ctoi.remove('T_DP(P)')
ctoi.extend(adata.obs[adata.obs['taa_l1'].isin(['TEC', #'Fb', 'EC', 'B', 'Mural', 'Mac', 'Mono', 'DC'
                                                ])]['taa_l4'].unique().tolist())

np.array(ctoi)

In [ ]:
# Subset adata
adata_sub = adata[adata.obs['taa_l4'].isin(ctoi),:].copy()
adata_sub.obs['cell_type'] = adata_sub.obs.apply(lambda x: x['taa_l4'] if x['taa_l1'] in ['T', 'B'] else x['taa_l3'], axis = 1)

# Remove B_dev (very few cells)
adata_sub = adata_sub[adata_sub.obs['cell_type'] != 'B_dev',:]

adata_sub.obs['cell_type'].value_counts()

In [ ]:
# Log-normalise data
sc.pp.normalize_total(adata_sub, target_sum=1e4)
sc.pp.log1p(adata_sub)

In [ ]:
interactions = li.rs.select_resource('cellphonedb')

# Select interacrions of interest
lroi = ['CCR7', 'CCR9','CD69', 'S1PR1', 'CCR4', 'LTB', 'CXCR4']
interactions_oi = interactions[(interactions['ligand'].isin(lroi)) | (interactions['receptor'].isin(lroi))]
interactions_oi = list(interactions_oi.itertuples(index=False, name=None))

interactions_oi[:5]

In [ ]:
moi = list(set([i[0] for i in interactions_oi] + [i[1] for i in interactions_oi]))
np.array(moi)

In [ ]:
# Filter out donors with fewer than 100 cells
min_cells = 100
ncells_by_sample = adata_sub.obs[col_sample].value_counts().to_frame(name = 'n_cells')
print('Removing {} donors with fewer than {} cells'.format(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].shape[0], min_cells))
adata_sub = adata_sub[~adata_sub.obs[col_sample].isin(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].index)]

In [ ]:
# Level to cell type assignment
ct_anno_levels = adata_sub.obs[['cell_type', 'taa_l4', 'taa_l3', 'taa_l1']].drop_duplicates().copy()
ct_anno_levels['level'] = ct_anno_levels.apply(lambda x: 'taa_l4' if x['taa_l1'] in ['T', 'B'] else 'taa_l3', axis = 1)
ct_anno_levels = ct_anno_levels.groupby('level').agg(list)
ct_anno_levels = ct_anno_levels.to_dict()['cell_type']

taa_l4_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_fine}_byDonor_freq.csv', index_col = 0)
taa_l4_freq = taa_l4_freq.loc[taa_l4_freq[col_cell_type_fine].isin(ct_anno_levels['taa_l4']),:].rename(columns = {col_cell_type_fine: 'cell_type'})
taa_l3_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_broad}_byDonor_freq.csv', index_col = 0)
taa_l3_freq = taa_l3_freq.loc[taa_l3_freq[col_cell_type_broad].isin(ct_anno_levels['taa_l3']),:].rename(columns = {col_cell_type_broad: 'cell_type'})

ct_freq = pd.concat([taa_l4_freq, taa_l3_freq], axis = 0)
ct_freq.head()

In [ ]:
import pickle 

taa_l4_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_adult_vs_infant_taa_l4_ageEffect.pkl'
taa_l3_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_taa_l3_adult_vs_infant_ageEffect.pkl'

with open(taa_l4_degs, 'rb') as f:
    taa_l4_degs = pickle.load(f)
taa_l4_degs = {k:v for k,v in taa_l4_degs.items() if k in ct_anno_levels['taa_l4']}
taa_l4_degs = pd.concat(taa_l4_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

with open(taa_l3_degs, 'rb') as f:
    taa_l3_degs = pickle.load(f)
taa_l3_degs = {k:v for k,v in taa_l3_degs.items() if k in ct_anno_levels['taa_l3']}
taa_l3_degs = pd.concat(taa_l3_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

all_degs = pd.concat([taa_l4_degs, taa_l3_degs])
all_degs = all_degs.loc[all_degs.index.isin(moi)]
all_degs['is_signif'] = all_degs.apply(lambda x: (x['padj'] < 0.05) and (abs(x['log2FoldChange']) > 1), axis = 1)
all_degs['is_signif'] = all_degs['is_signif'].apply(lambda x: '*' if x else '')

all_degs.head()

In [ ]:
import matplotlib
df = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'log2FoldChange')
df_annot = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'is_signif', aggfunc = 'first', fill_value='')

# Convert df_annot to contain only numerical values or properly formatted strings
df_annot = df_annot.applymap(lambda x: '*' if x == '*' else '')

plt.figure(figsize = calc_figsize(width = 80, height = 60))
p = sns.heatmap(df, cmap='PuOr_r', center=0, vmin=-10, vmax=10, cbar_kws={'label': 'log2FC'}, xticklabels=True, yticklabels=True,
                annot=np.array(df_annot), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

for t in p.texts:
    trans = t.get_transform()
    offs = matplotlib.transforms.ScaledTranslation(0, 0,
                    matplotlib.transforms.IdentityTransform())
    t.set_transform( offs + trans )
    
p.set_xlabel('Cell type')
p.set_xticklabels(p.get_xticklabels(), rotation = 90)
p.set_ylabel('Gene')
p.set_yticklabels(p.get_yticklabels(), rotation = 0)
p.figure.tight_layout()
#p.figure.set_size_inches(calc_figsize(width = 80, height = 60))
plt.savefig(f'{plots_path}/cellComm/medulla/thyAgeing_diffCCC_medulla_degs_heatmap.pdf', bbox_inches = 'tight')

In [ ]:
lr_res = li.multi.df_to_lr(adata_sub[adata_sub.obs[col_age_group] == 'adult'],
                           dea_df=all_degs,
                           resource_name='consensus', 
                           interactions=interactions_oi,
                           expr_prop=0.1, # calculated for adata as passed - used to filter interactions
                           groupby='cell_type',
                           stat_keys=['log2FoldChange','stat', 'pvalue', 'padj'],
                           use_raw=False,
                           complex_col='stat', # NOTE: we use the Wald Stat to deal with complexes
                           verbose=True,
                           return_all_lrs=True,
                           )

In [ ]:
lr_res.to_csv(f'{data_path}/analyses/cellComm/medulla/thyAgeing_diffCCC_medulla_allSignalling.csv', index = False)
lr_res.head()

In [ ]:
lr_res

In [ ]:
ia_list = lr_res.interaction.unique().tolist()
sources = ['cTEC','mcTEC', 'mTECI', 'mTECII', 'mTECIII', 'TEC-EMT', 'TEC-mim']
targets = ['T_DP(Q)','T_αβT(entry)','T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg']
df = lr_res.loc[lr_res['interaction'].isin(ia_list)].copy()
df_ligand = df.loc[df['source'].str.contains('TEC')].pivot_table(index = 'interaction', columns = 'source', values = 'ligand_log2FoldChange').loc[ia_list, sources]
df_ligand_anno = df.loc[df['source'].str.contains('TEC')].pivot_table(index = 'interaction', columns = 'source', values = 'ligand_padj').applymap(lambda x: '*' if x < .05 else '').loc[ia_list, sources]
df_rec = df.loc[~df['target'].str.contains('TEC')].pivot_table(index = 'interaction', columns = 'target', values = 'receptor_log2FoldChange').loc[ia_list, targets]
df_rec_anno = df.loc[~df['target'].str.contains('TEC')].pivot_table(index = 'interaction', columns = 'target', values = 'receptor_padj').applymap(lambda x: '*' if x < .05 else '').loc[ia_list, targets]

In [ ]:
from matplotlib import colors as mcolors
max_abs = np.array([abs(df_ligand.dropna().values).max(), abs(df_rec.dropna().values).max()]).max()

width_ratios = [len(sources), len(targets)]
fig, axes = plt.subplots(1,2, figsize=calc_figsize(width=60, height=20), 
                            gridspec_kw={'width_ratios': width_ratios, 'wspace': 0.05}
                            )

# Color map
cmap = sns.color_palette('PuOr_r', as_cmap=True)
norm = mcolors.TwoSlopeNorm(vcenter=0, vmin=-max_abs, vmax=max_abs)

sns.heatmap(df_ligand, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[0],
                annot=np.array(df_ligand_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[0].set_title(f'Ligand', fontweight='bold')
axes[0].set_xlabel('Cell population')
axes[0].set_ylabel('Interaction')
axes[0].tick_params(axis='x', rotation=90)
axes[0].tick_params(axis='y', rotation=0)

sns.heatmap(df_rec, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[1],
                annot=np.array(df_rec_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[1].set_title(f'Receptor', fontweight='bold')
axes[1].set_xlabel('Cell population')
axes[1].tick_params(axis='x', rotation=90)
axes[1].set_ylabel('')
axes[1].set_yticklabels([])
axes[1].set_yticks([])

 # Add a colorbar to the right of the last heatmap
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='vertical', fraction=0.05, pad=0.0)
cbar.set_label('Log2FC', rotation=270, labelpad=5)
fig.subplots_adjust(right=0.85)

plt.savefig(f'{plots_path}/cellComm/medulla/thyAgeing_diffCCC_medulla_ia_heatmap.pdf')

### All migration

In [ ]:
# List cell types of interest
ctoi = adata.obs[adata.obs['taa_l2'].isin(['T_predev','T_dev', 'T_mature'])]['taa_l4'].unique().tolist() 
ctoi.extend(adata.obs[adata.obs['taa_l1'].isin(['TEC', 'Fb', 'EC', 'B', 'Mural', 'Mac', 'Mono', 'DC'
                                                ])]['taa_l4'].unique().tolist())

np.array(ctoi)

In [ ]:
# Subset adata
adata_sub = adata[adata.obs['taa_l4'].isin(ctoi),:].copy()
adata_sub.obs['cell_type'] = adata_sub.obs.apply(lambda x: x['taa_l4'] if x['taa_l1'] in ['T', 'B'] else x['taa_l3'], axis = 1)

# Remove B_dev (very few cells)
adata_sub = adata_sub[adata_sub.obs['cell_type'] != 'B_dev',:]

adata_sub.obs['cell_type'].value_counts()

In [ ]:
# Log-normalise data
sc.pp.normalize_total(adata_sub, target_sum=1e4)
sc.pp.log1p(adata_sub)

In [ ]:
interactions = li.rs.select_resource('cellphonedb')

# Select interacrions of interest
lroi = ['CCR7', 'CCR9','CD69', 'S1PR1', 'CCR4', 'LTB', 'CXCR4'] + ['NOTCH1', 'LTB', 'LTA', 'IL7', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'IL7R', 'CXCR4', 'DLK2']
interactions_oi = interactions[(interactions['ligand'].isin(lroi)) | (interactions['receptor'].isin(lroi))]
interactions_oi = list(interactions_oi.itertuples(index=False, name=None))

interactions_oi[:5]

In [ ]:
moi = list(set([i[0] for i in interactions_oi] + [i[1] for i in interactions_oi]))
np.array(moi)

In [ ]:
# Filter out donors with fewer than 100 cells
min_cells = 100
ncells_by_sample = adata_sub.obs[col_sample].value_counts().to_frame(name = 'n_cells')
print('Removing {} donors with fewer than {} cells'.format(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].shape[0], min_cells))
adata_sub = adata_sub[~adata_sub.obs[col_sample].isin(ncells_by_sample[ncells_by_sample['n_cells'] < min_cells].index)]

In [ ]:
# Level to cell type assignment
ct_anno_levels = adata_sub.obs[['cell_type', 'taa_l4', 'taa_l3', 'taa_l1']].drop_duplicates().copy()
ct_anno_levels['level'] = ct_anno_levels.apply(lambda x: 'taa_l4' if x['taa_l1'] in ['T', 'B'] else 'taa_l3', axis = 1)
ct_anno_levels = ct_anno_levels.groupby('level').agg(list)
ct_anno_levels = ct_anno_levels.to_dict()['cell_type']

taa_l4_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_fine}_byDonor_freq.csv', index_col = 0)
taa_l4_freq = taa_l4_freq.loc[taa_l4_freq[col_cell_type_fine].isin(ct_anno_levels['taa_l4']),:].rename(columns = {col_cell_type_fine: 'cell_type'})
taa_l3_freq = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_{col_cell_type_broad}_byDonor_freq.csv', index_col = 0)
taa_l3_freq = taa_l3_freq.loc[taa_l3_freq[col_cell_type_broad].isin(ct_anno_levels['taa_l3']),:].rename(columns = {col_cell_type_broad: 'cell_type'})

ct_freq = pd.concat([taa_l4_freq, taa_l3_freq], axis = 0)
ct_freq.head()

In [ ]:
import pickle 

taa_l4_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_adult_vs_infant_taa_l4_ageEffect.pkl'
taa_l3_degs = f'{general_data_path}/analyses/dea/thyAgeing_dea_taa_l3_adult_vs_infant_ageEffect.pkl'

with open(taa_l4_degs, 'rb') as f:
    taa_l4_degs = pickle.load(f)
taa_l4_degs = {k:v for k,v in taa_l4_degs.items() if k in ct_anno_levels['taa_l4']}
taa_l4_degs = pd.concat(taa_l4_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

with open(taa_l3_degs, 'rb') as f:
    taa_l3_degs = pickle.load(f)
taa_l3_degs = {k:v for k,v in taa_l3_degs.items() if k in ct_anno_levels['taa_l3']}
taa_l3_degs = pd.concat(taa_l3_degs).reset_index(names = ['cell_type','gene_name']).set_index('gene_name')

all_degs = pd.concat([taa_l4_degs, taa_l3_degs])
all_degs = all_degs.loc[all_degs.index.isin(moi)]
all_degs['is_signif'] = all_degs.apply(lambda x: (x['padj'] < 0.05) and (abs(x['log2FoldChange']) > 1), axis = 1)
all_degs['is_signif'] = all_degs['is_signif'].apply(lambda x: '*' if x else '')

all_degs.head()

In [ ]:
import matplotlib
df = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'log2FoldChange')
df_annot = all_degs.reset_index(names = 'gene_name').pivot_table(index = 'cell_type', columns = 'gene_name', values = 'is_signif', aggfunc = 'first', fill_value='')

# Convert df_annot to contain only numerical values or properly formatted strings
df_annot = df_annot.applymap(lambda x: '*' if x == '*' else '')

plt.figure(figsize = calc_figsize(width = 80, height = 60))
p = sns.heatmap(df, cmap='PuOr_r', center=0, vmin=-10, vmax=10, cbar_kws={'label': 'log2FC'}, xticklabels=True, yticklabels=True,
                annot=np.array(df_annot), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

for t in p.texts:
    trans = t.get_transform()
    offs = matplotlib.transforms.ScaledTranslation(0, 0,
                    matplotlib.transforms.IdentityTransform())
    t.set_transform( offs + trans )
    
p.set_xlabel('Cell type')
p.set_xticklabels(p.get_xticklabels(), rotation = 90)
p.set_ylabel('Gene')
p.set_yticklabels(p.get_yticklabels(), rotation = 0)
p.figure.tight_layout()
#p.figure.set_size_inches(calc_figsize(width = 80, height = 60))
plt.savefig(f'{plots_path}/cellComm/medulla/thyAgeing_diffCCC_medulla_degs_heatmap.pdf', bbox_inches = 'tight')

In [ ]:
lr_res = li.multi.df_to_lr(adata_sub[adata_sub.obs[col_age_group] == 'adult'],
                           dea_df=all_degs,
                           resource_name='consensus', 
                           interactions=interactions_oi,
                           expr_prop=0.1, # calculated for adata as passed - used to filter interactions
                           groupby='cell_type',
                           stat_keys=['log2FoldChange','stat', 'pvalue', 'padj'],
                           use_raw=False,
                           complex_col='stat', # NOTE: we use the Wald Stat to deal with complexes
                           verbose=True,
                           return_all_lrs=True,
                           )

In [ ]:
import re
ia_list = sorted(lr_res.interaction.unique().tolist())
df = lr_res.loc[lr_res['interaction'].isin(ia_list)].copy().dropna(subset = ['ligand_log2FoldChange', 'receptor_log2FoldChange'])
df = df.loc[~df['receptor'].isin(['NOTCH2', 'NOTCH3', 'NOTCH4'])]
ia_list = sorted(df.interaction.unique().tolist())

sources = df.loc[~df['source'].str.startswith('T_'), 'source'].unique().tolist()
df_ligand = df.loc[df['source'].isin(sources)].pivot_table(index = 'interaction', columns = 'source', values = 'ligand_log2FoldChange').loc[ia_list, sources]
df_ligand_anno = df.loc[df['source'].isin(sources)].pivot_table(index = 'interaction', columns = 'source', values = 'ligand_padj').applymap(lambda x: '*' if x < .05 else '').loc[ia_list, sources]

targets = [c for c in ctoi if c in df.loc[df['target'].str.startswith('T_'), 'target'].unique().tolist()]
df_rec = df.loc[df['target'].isin(targets)].pivot_table(index = 'interaction', columns = 'target', values = 'receptor_log2FoldChange').loc[ia_list, targets]
df_rec_anno = df.loc[df['target'].isin(targets)].pivot_table(index = 'interaction', columns = 'target', values = 'receptor_padj').applymap(lambda x: '*' if x < .05 else '').loc[ia_list, targets]

In [ ]:
from matplotlib import colors as mcolors
max_abs = max(abs(np.nan_to_num(df_ligand.values.flatten(), nan=0)).max(), abs(np.nan_to_num(df_rec.values.flatten(), nan=0)).max())
max_abs = 5

width_ratios = [len(sources), len(targets)]
fig, axes = plt.subplots(1,2, figsize=calc_figsize(width='double', height=80), 
                            gridspec_kw={'width_ratios': width_ratios, 'wspace': 0.05}
                            )

# Color map
cmap = sns.color_palette('PuOr_r', as_cmap=True)
norm = mcolors.TwoSlopeNorm(vcenter=0, vmin=-max_abs, vmax=max_abs)

sns.heatmap(df_ligand, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[0],
                annot=np.array(df_ligand_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[0].set_title(f'Ligand', fontweight='bold')
axes[0].set_xlabel('Cell population')
axes[0].set_ylabel('Interaction')
axes[0].tick_params(axis='x', rotation=90)
axes[0].tick_params(axis='y', rotation=0)

sns.heatmap(df_rec, cmap=cmap, norm = norm, cbar=False, xticklabels=True, yticklabels=True, ax=axes[1],
                annot=np.array(df_rec_anno), fmt="", annot_kws={'fontsize': 10, 'va': 'top', 'ha': 'center', 'ma' : 'center'})

axes[1].set_title(f'Receptor', fontweight='bold')
axes[1].set_xlabel('Cell population')
axes[1].tick_params(axis='x', rotation=90)
axes[1].set_ylabel('')
axes[1].set_yticklabels([])
axes[1].set_yticks([])

 # Add a colorbar to the right of the last heatmap
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='vertical', fraction=0.05, pad=0.0)
cbar.set_label('Log2FC', rotation=270, labelpad=5)
fig.subplots_adjust(right=0.85)

plt.savefig(f'{plots_path}/cellComm/medulla/thyAgeing_diffCCC_allMigration_ia_heatmap.pdf')